# Finch usage

Finch is a WPS server for climate indicators, but also has a few utilities to facilitate data handling. To get started, first instantiate the client. 

In [ ]:
from __future__ import annotations

from birdy import WPSClient

url = "https://finch.crim.ca/wps"
# url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/finch/wps'
# url = 'http://localhost:5000'
wps = WPSClient(url)

The list of available processes can be displayed using the help function, as well as details about individual processes. 

In [ ]:
?wps

In [ ]:
?wps.frost_days

To actually compute an indicator, we need to specify the path to the netCDF file used as input for the calculation of the indicator. To compute `frost_days`, we need a time series of daily minimum temperature. Here we'll use a small test file. Note that here we're using an OPeNDAP link, but it could also be an url to a netCDF file, or the path to a local file on disk. We then simply call the indicator. The response is an object that can poll the server to inquire about the status of the process. This object can use two modes: 
 - synchronous: it will wait for the server's response before returning; or 
 - asynchronous: it will return immediately, but without the actual output from the process.
 
Here, since we're applying the process on a small test file, we're using the default synchronous mode. For long computations, use the asynchronous mode to avoid time-out errors. The asynchronous mode is activated by setting the `progress` attribute of the WPS client to True. 

In [ ]:
tasmin = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/testdata/flyingpigeon/cmip3/tasmin.sresa2.miub_echo_g.run1.atm.da.nc"
resp = wps.frost_days(tasmin)

In [ ]:
print(resp.status)
out = resp.get()
print(out)

The `get` method returns a `NamedTuple` object with all the WPS outputs, either as references to files or actual content. To copy the file to the local disk, you can use the `getOutput` method. 

In [ ]:
resp.getOutput("/tmp/out.nc")

In [ ]:
import xarray as xr

xr.open_dataset("/tmp/out.nc")

The birdy client offers a quicker way to download and open the files automatically using `asobj=True`, as long as the file format is known to birdy. 

In [ ]:
ds, log = resp.get(asobj=True)

In [ ]:
ds

In [ ]:
print(log)